<a href="https://colab.research.google.com/github/tejusn/doc-ai-exp/blob/main/Learn_Docling_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
!pip install docling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.4/251.4 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.5/164.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 94.4 MB/s eta 0:00:00
   ━━

## Document Conversion

In [4]:
from docling.document_converter import DocumentConverter
from IPython.display import display
import json

source = "https://arxiv.org/pdf/2501.17887"
# source = "/content/sample_data/DocLing_2501.17887v1.pdf"
converter = DocumentConverter()
doc = converter.convert(source).document

print(doc.export)

[INFO] 2025-10-26 17:10:37,278 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-10-26 17:10:37,347 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2025-10-26 17:10:37,350 [RapidOCR] torch.py:54: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2025-10-26 17:10:37,633 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-10-26 17:10:37,640 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2025-10-26 17:10:37,641 [RapidOCR] torch.py:54: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2025-10-26 17:10:37,752 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-10-26 17:10:37,878 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/di

<bound method DoclingDocument.export_to_document_tokens of DoclingDocument(schema_name='DoclingDocument', version='1.7.0', name='2501.17887v1', origin=DocumentOrigin(mimetype='application/pdf', binary_hash=1554916940463874186, filename='2501.17887v1.pdf', uri=None), furniture=GroupItem(self_ref='#/furniture', parent=None, children=[], content_layer=<ContentLayer.FURNITURE: 'furniture'>, name='_root_', label=<GroupLabel.UNSPECIFIED: 'unspecified'>), body=GroupItem(self_ref='#/body', parent=None, children=[RefItem(cref='#/texts/0'), RefItem(cref='#/texts/1'), RefItem(cref='#/texts/2'), RefItem(cref='#/texts/3'), RefItem(cref='#/texts/4'), RefItem(cref='#/texts/5'), RefItem(cref='#/texts/6'), RefItem(cref='#/texts/7'), RefItem(cref='#/texts/8'), RefItem(cref='#/texts/9'), RefItem(cref='#/texts/10'), RefItem(cref='#/texts/11'), RefItem(cref='#/texts/12'), RefItem(cref='#/texts/13'), RefItem(cref='#/groups/0'), RefItem(cref='#/texts/20'), RefItem(cref='#/texts/21'), RefItem(cref='#/pictures

### Export methods of the DoclingDocument

In [6]:
print(doc.export_to_dict())
# print(doc.export_to_html())
# print(doc.export_to_doctags())
# print(doc.export_to_markdown())

{'schema_name': 'DoclingDocument',
 'version': '1.7.0',
 'name': '2501.17887v1',
 'origin': {'mimetype': 'application/pdf',
  'binary_hash': 1554916940463874186,
  'filename': '2501.17887v1.pdf'},
 'furniture': {'self_ref': '#/furniture',
  'children': [],
  'content_layer': 'furniture',
  'name': '_root_',
  'label': 'unspecified'},
 'body': {'self_ref': '#/body',
  'children': [{'$ref': '#/texts/0'},
   {'$ref': '#/texts/1'},
   {'$ref': '#/texts/2'},
   {'$ref': '#/texts/3'},
   {'$ref': '#/texts/4'},
   {'$ref': '#/texts/5'},
   {'$ref': '#/texts/6'},
   {'$ref': '#/texts/7'},
   {'$ref': '#/texts/8'},
   {'$ref': '#/texts/9'},
   {'$ref': '#/texts/10'},
   {'$ref': '#/texts/11'},
   {'$ref': '#/texts/12'},
   {'$ref': '#/texts/13'},
   {'$ref': '#/groups/0'},
   {'$ref': '#/texts/20'},
   {'$ref': '#/texts/21'},
   {'$ref': '#/pictures/0'},
   {'$ref': '#/texts/64'},
   {'$ref': '#/texts/65'},
   {'$ref': '#/texts/66'},
   {'$ref': '#/texts/67'},
   {'$ref': '#/texts/68'},
   {'$r

## Summarize DoclingDocument Structure

In [ ]:
import json

# Create a dictionary with key information from the doc object
doc_summary = {
    "schema_name": doc.schema_name,
    "version": doc.version,
    "name": doc.name,
    "origin": doc.origin.model_dump() if doc.origin else None, # Include origin details
    "furniture_item_count": len(doc.furniture.children) if doc.furniture and hasattr(doc.furniture, 'children') else 0, # Count furniture items
    "body_item_count": len(doc.body.children) if doc.body and hasattr(doc.body, 'children') else 0, # Count body items
    "text_count": len(doc.texts), # Count text items
    "table_count": len(doc.tables), # Count table items
    "picture_count": len(doc.pictures), # Count picture items
    "group_count": len(doc.groups), # Count group items
    # You can add other top-level attributes if needed
    # "metadata": doc.metadata.model_dump() if doc.metadata else None,
}

# Convert the dictionary to a JSON string with indentation for pretty printing
doc_json_summary = json.dumps(doc_summary, indent=4)

print(doc_json_summary)

### Exploring Text Elements

In [ ]:
display(doc.texts[0].model_dump())

In [ ]:
print("Exploring Text Elements:")
# Iterate through the text items in the document
for i, text_item in enumerate(doc.texts[:10]): # Limit to the first 10 items
    print(f"Text Item {i}:")
    print(f"  Label: {text_item.label}")
    # Print the first 200 characters of the text to avoid overly long output
    print(f"  Text: {text_item.text[:200]}...")
    if hasattr(text_item, 'prov'):
        print(f"  Provenance (first item): Page {text_item.prov[0].page_no}, Bbox: {text_item.prov[0].bbox}")
    print("-" * 20)

### Exploring Table Elements

In [ ]:
print("Exploring Table Elements:")
# Iterate through the table items in the document
for i, table_item in enumerate(doc.tables):
    print(f"Table Item {i}:")
    print(f"  Label: {table_item.label}")
    if hasattr(table_item, 'prov'):
        print(f"  Provenance (first item): Page {table_item.prov[0].page_no}, Bbox: {table_item.prov[0].bbox}")

    # Displaying table data can be complex, here's a simple representation
    if hasattr(table_item, 'data') and hasattr(table_item.data, 'table_cells'):
        print(f"  Number of cells: {len(table_item.data.table_cells)}")
        # You can add code here to extract and display table data more formally (like a DataFrame)
        # as shown in the previous examples.
    print("-" * 20)

### Exploring Picture Elements

In [ ]:
print("Exploring Picture Elements:")
# Iterate through the picture items in the document
for i, picture_item in enumerate(doc.pictures):
    print(f"Picture Item {i}:")
    print(f"  Label: {picture_item.label}")
    if hasattr(picture_item, 'prov'):
        print(f"  Provenance (first item): Page {picture_item.prov[0].page_no}, Bbox: {picture_item.prov[0].bbox}")
    # You can add code here to display the image as shown in the previous examples.
    print("-" * 20)

## Visualize Bounding Boxes

In [ ]:
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import io # Import the io module for handling byte streams

# Assuming the document is a PDF, you might need a way to get an image of a specific page.
# Docling might provide a method for this, or you might need another library like pypdfium2.
# For demonstration, let's assume you have an image file of a page.
# Replace 'path/to/your/page_image.png' with the actual path to your image file.
# If the source is a PDF, you can use pypdfium2 to render a page to an image.

# Example using pypdfium2 (install with !pip install pypdfium2)
import pypdfium2 as pdfium

# Load the PDF document
pdf = pdfium.PdfDocument(source)

# --- Specify the page number here (0-indexed) ---
page_index = 4
# -------------------------------------------------

page = pdf.get_page(page_index)

# Render the page to a PdfBitmap and then convert to PIL Image using to_pil()
bitmap = page.render(scale=2) # Scale up for better resolution
pil_image = bitmap.to_pil()


# Create a drawing object
draw = ImageDraw.Draw(pil_image)

# Iterate through the elements in the document and draw their bounding boxes
# You can choose which elements to visualize (e.g., texts, tables, figures)
# We will check the provenance to ensure the element is on the selected page
for item in doc.texts + doc.tables + doc.pictures + doc.groups: # Include all relevant item types
    if hasattr(item, 'prov'): # Check if the item has provenance information
        for prov_item in item.prov:
            if prov_item.page_no == page_index + 1: # Check if the provenance item is on the current page (page_no is 1-indexed)
                bbox = prov_item.bbox
                # Convert the bbox coordinates to the image's coordinate system (top-left origin)
                # Docling uses BOTTOMLEFT, Pillow uses TOPLEFT
                img_width, img_height = pil_image.size

                # Assuming bbox coordinates are relative to the bottom-left of the page and scaled to 1
                # You might need to scale the bbox coordinates by the image dimensions
                # Example:
                page_width = page.get_width()
                page_height = page.get_height()
                scale_x = img_width / page_width
                scale_y = img_height / page_height

                top_left_x = bbox.l * scale_x
                top_left_y = img_height - (bbox.t * scale_y) # Flip y-axis
                bottom_right_x = bbox.r * scale_x
                bottom_right_y = img_height - (bbox.b * scale_y) # Flip y-axis


                # Draw the rectangle
                # You can customize the color and width based on item type if you want
                color = "red"
                if item in doc.tables:
                    color = "blue"
                elif item in doc.pictures:
                    color = "green"
                elif item in doc.groups:
                    color = "purple"

                draw.rectangle([top_left_x, top_left_y, bottom_right_x, bottom_right_y], outline=color, width=2)

# Display the image
plt.figure(figsize=(10, 10))
plt.imshow(pil_image)
plt.axis('off')
plt.show()

# Don't forget to close the PDF document
pdf.close()

## Extract Images and Charts with Associated Text

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import pypdfium2 as pdfium # Import pypdfium2

# Get the page index and the rendered image from the previous visualization cell
# Make sure to run the visualization cell first to set these variables
try:
    page_to_extract = page_index + 1 # Convert 0-indexed page_index to 1-indexed page_no
    pil_image_page = pil_image # Use the PIL image of the page from the previous cell
    # Re-obtain the page object within this cell
    pdf = pdfium.PdfDocument(source) # Load the PDF again
    page = pdf.get_page(page_index) # Get the specific page
except NameError:
    print("Please run the visualization cell first to define 'page_index' and generate 'pil_image'.")
    page_to_extract = None
    pil_image_page = None
    page = None # Ensure page is None if previous variables are not set

if page_to_extract is not None and pil_image_page is not None and page is not None:
    print(f"Extracting images and charts from page {page_to_extract}:")
    for item in doc.pictures:
        # Check if the item is on the specified page
        item_on_page = False
        if hasattr(item, 'prov'):
            for prov_item in item.prov:
                if prov_item.page_no == page_to_extract:
                    item_on_page = True
                    bbox = prov_item.bbox # Get the bounding box
                    break # Found provenance on the target page, no need to check further

        if item_on_page:
            print(f"\nFound image or chart:")
            if bbox:
                # Convert the bbox coordinates to the image's coordinate system (top-left origin)
                # Docling uses BOTTOMLEFT, Pillow uses TOPLEFT
                img_width, img_height = pil_image_page.size
                page_width = page.get_width()
                page_height = page.get_height()
                scale_x = img_width / page_width
                scale_y = img_height / page_height

                top_left_x = bbox.l * scale_x
                top_left_y = img_height - (bbox.t * scale_y) # Flip y-axis
                bottom_right_x = bbox.r * scale_x
                bottom_right_y = img_height - (bbox.b * scale_y) # Flip y-axis

                # Crop the image using the bounding box
                try:
                    cropped_img = pil_image_page.crop((top_left_x, top_left_y, bottom_right_x, bottom_right_y))
                    display(cropped_img)

                    # Attempt to find and print associated text
                    associated_text = []
                    if hasattr(item, 'children'):
                        for child_ref in item.children:
                            # Assuming children are text references
                            if child_ref.cref.startswith('#/texts/'):
                                text_index = int(child_ref.cref.split('/')[-1])
                                if text_index < len(doc.texts):
                                    associated_text.append(doc.texts[text_index].text)
                    # Also check for captions attribute if available
                    if hasattr(item, 'captions'):
                        for caption_ref in item.captions:
                              if caption_ref.cref.startswith('#/texts/'):
                                text_index = int(caption_ref.cref.split('/')[-1])
                                if text_index < len(doc.texts):
                                    associated_text.append(doc.texts[text_index].text)

                    if associated_text:
                        print("Associated Text:")
                        for text in associated_text:
                            print(text)
                    else:
                        print("No directly associated text found in children or captions.")


                except Exception as e:
                    print(f"Could not crop or display image: {e}")
            else:
                print("No bounding box found for this image item.")

    # Don't forget to close the PDF document
    pdf.close()
else:
    print("Could not proceed with image extraction because required variables or page object were not available.")

## Extract Table Data

In [ ]:
import pandas as pd

# Get the page index from the previous cell
# Make sure to run the previous cell first to set the page_index variable
try:
    page_to_extract = page_index + 1 # Convert 0-indexed page_index to 1-indexed page_no
except NameError:
    print("Please run the previous cell first to define 'page_index'.")
    page_to_extract = None

if page_to_extract is not None:
    print(f"Extracting tables from page {page_to_extract}:")
    for table_item in doc.tables:
        # Check if the table is on the specified page
        table_on_page = False
        if hasattr(table_item, 'prov'):
            for prov_item in table_item.prov:
                if prov_item.page_no == page_to_extract:
                    table_on_page = True
                    break # Found provenance on the target page, no need to check further

        if table_on_page:
            print(f"\nFound table:")
            # Extract table data into a list of lists
            table_data = []
            if hasattr(table_item, 'data') and hasattr(table_item.data, 'table_cells'):
                # Sort cells by row and then by column for correct order
                sorted_cells = sorted(table_item.data.table_cells, key=lambda cell: (cell.start_row_offset_idx, cell.start_col_offset_idx))

                # Determine the number of rows and columns
                max_row = max(cell.end_row_offset_idx for cell in sorted_cells) if sorted_cells else 0
                max_col = max(cell.end_col_offset_idx for cell in sorted_cells) if sorted_cells else 0

                # Initialize table_data with empty strings
                table_data = [['' for _ in range(max_col)] for _ in range(max_row)]

                # Fill table_data with cell text, considering row and column spans
                for cell in sorted_cells:
                    for r in range(cell.start_row_offset_idx, cell.end_row_offset_idx):
                        for c in range(cell.start_col_offset_idx, cell.end_col_offset_idx):
                            if r < max_row and c < max_col: # Ensure within bounds
                                table_data[r][c] = cell.text

            # Display the table data, for example as a pandas DataFrame
            if table_data:
                try:
                    df = pd.DataFrame(table_data)
                    display(df)
                except Exception as e:
                    print(f"Could not display table as DataFrame: {e}")
                    print("Raw table data:")
                    print(table_data)
            else:
                print("No cells found for this table.")